# All in one

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [5]:
from crewai import Agent, Task, Crew

In [6]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

In [7]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [8]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [9]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [14]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [15]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [16]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [17]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d03ade2d-bde2-4462-aa08-19a7a026b71d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b7436995-fe2c-4129-b24c-8a31a18823a0
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: b7436995-fe2c-4129-b24c-8a31a18823a0
│      Status: Executing Task...
└── 📋 Task: dc290564-34d4-4d6a-af01-0db604715209
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: b7436995-fe2c-4129-b24c-8a31a18823a0
│      Status: Executing Task...
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
└── 📋 Task: dc290564-34d4-4d6a-af01-0db604715209
       Status: Executing Task...

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


...



# Agent: Engineering Interview Preparer
## Final Answer: 
Interview Questions and Talking Points:
1. Can you provide examples of how you have successfully led and managed engineering teams in your previous roles?
2. How have you demonstrated your ability to scale engineering teams while maintaining high performance standards?
3. Discuss a specific project where you effectively implemented scalable AI solutions. What were the challenges you faced and how did you overcome them?
4. In what ways have you contributed to building a culture of innovation within your engineering teams?
5. How do you approach mentorship and leadership development within your team?
6. Can you share a situation where you had to make critical decisions under pressure? How did you handle it?
7. What methodologies or frameworks do you prefer when it comes to software development and team collaboration?
8. Have you worked on remote teams before? How do you ensure effective communication and collaboration in a remot

🚀 Crew: crew
├── 📋 Task: b7436995-fe2c-4129-b24c-8a31a18823a0
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
│       └── 🔧 Used Read website content (2)
├── 📋 Task: dc290564-34d4-4d6a-af01-0db604715209
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (2)
├── 📋 Task: 19b1e274-43fb-41d7-9d5b-943d6482c7ec
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
└── 📋 Task: 4f30636d-0603-4ead-8cfb-08612a668e34
       Status: Executing Task...
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b7436995-fe2c-4129-b24c-8a31a18823a0
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
│       └── 🔧 Used Read website content (2)
├── 📋 Task: dc290564-34d4-4d6a-af01-0db604715209
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (2)
├── 📋 Task: 19b1e274-43fb-41d7-9d5b-943d6482c7ec
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
└── 📋 Task: 4f30636d-0603-4ead-8cfb-08612a668e34
       Assigned to: Engineering Interview Preparer
       Status: ✅ Completed
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4f30636d-0603-4ead-8cfb-08612a668e34                                                                     │
│  Agent: Engineering Interview Preparer                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d03ade2d-bde2-4462-aa08-19a7a026b71d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [18]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

Due to access limitations, we can craft a tailored resume based on the existing information about João Moura as a Principal Software Engineer and Senior Engineering Manager. We will proceed to update the resume to highlight his relevant experiences, accomplishments, and skills.

In [19]:
display(Markdown("./interview_materials.md"))

Interview Questions and Talking Points:
1. Can you provide examples of how you have successfully led and managed engineering teams in your previous roles?
2. How have you demonstrated your ability to scale engineering teams while maintaining high performance standards?
3. Discuss a specific project where you effectively implemented scalable AI solutions. What were the challenges you faced and how did you overcome them?
4. In what ways have you contributed to building a culture of innovation within your engineering teams?
5. How do you approach mentorship and leadership development within your team?
6. Can you share a situation where you had to make critical decisions under pressure? How did you handle it?
7. What methodologies or frameworks do you prefer when it comes to software development and team collaboration?
8. Have you worked on remote teams before? How do you ensure effective communication and collaboration in a remote work environment?
9. Describe a time when you collaborated with cross-functional teams to achieve a common goal. What was the outcome?
10. How do you stay updated with industry trends and technologies to ensure your team remains competitive and innovative?

These questions and talking points will help the candidate highlight relevant experiences, accomplishments, and skills aligning with the job requirements as a Senior Engineering Manager or Principal Software Engineer.